In [37]:
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam


from UNet_model import ImageSeg
from sklearn.model_selection import train_test_split

import os
import numpy as np
import cv2
from random import randint

np.random.seed = 42

### Data Processing

In [3]:
# define input image size
img_H, img_W, img_C = 128, 128, 3

In [5]:
# define train & test directories
train_dir = 'data-science-bowl-2018/stage1_train/'
test_dir = 'data-science-bowl-2018/stage1_test/'

# get the full paths of train & test images
train_img_path = [] 
train_mask_path = []
for f in os.listdir(train_dir):
    train_img_path.append(train_dir + f + '/images/' + f + '.png')
    train_mask_path.append(train_dir + f + '/masks/')
test_img_path = [test_dir + f + '/images/' + f + '.png' for f in os.listdir(test_dir)]

In [6]:
def get_images(path):
    """ 
    the function reads, resizes the images & returns them in an arrays
    """
    images = []

    for img_path in path:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_H, img_W))
        images.append(img / 255.)
    
    return np.array(images)
    
    
def get_masks(path):
    """
    Masks of each image are in seperate images.
    The function combines all these masks into one image, 
    and returns it a boolean array
    """
    masks = []
    
    for mask_path in path:
        # create a black empty mask
        mask = np.zeros((img_H, img_W, 1), dtype= np.uint8)
        # loop through all masks of an image
        for m in os.listdir(mask_path):
            mask_ = cv2.imread(mask_path + m, 0)
            mask_ = cv2.resize(mask_, (img_H, img_W))
            mask_ = np.reshape(mask_, (img_H, img_W, 1))
            # get combine masks together by getting the maximum
            mask = np.maximum(mask, mask_)
            # convert into boolyean array
            mask = mask.astype(bool)

        masks.append(mask)
        
    return np.array(masks)

In [7]:
# get training data
x_train = get_images(train_img_path)
y_train = get_masks(train_mask_path)

# split train data into trani dataset & validation dataset
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1)

# get test data
x_test = get_images(test_img_path)

In [24]:
# check point
cv2.imshow('image', x_train[-1])
cv2.waitKey(0)

cv2.imshow('mask', np.uint8(np.where(y_train[-1] == True, 255, 0)))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Model

In [38]:
model = ImageSeg.UNet(img_H, img_W, img_C)
model.compile(Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_133 (Conv2D)             (None, 128, 128, 16) 448         input_8[0][0]                    
__________________________________________________________________________________________________
dropout_63 (Dropout)            (None, 128, 128, 16) 0           conv2d_133[0][0]                 
__________________________________________________________________________________________________
conv2d_134 (Conv2D)             (None, 128, 128, 16) 2320        dropout_63[0][0]                 
____________________________________________________________________________________________

In [40]:
# checkpoint
checkpoint = callbacks.ModelCheckpoint('Nuclei_model.h5', verbose=1, save_best_only=True)
callback = [checkpoint,
            callbacks.TensorBoard(log_dir='./logs')
           ]


results = model.fit(x_train,
                   y_train,
                   validation_data= (x_val, y_val),
                   batch_size=32,
                   epochs=25,
                   callbacks= callback)

Epoch 1/25
19/19 [==============================] - 113s 6s/step - loss: 0.5660 - accuracy: 0.8437 - val_loss: 0.5079 - val_accuracy: 0.8315

Epoch 00001: val_loss improved from inf to 0.50788, saving model to Nuclei_model.h5
Epoch 2/25
19/19 [==============================] - 92s 5s/step - loss: 0.3830 - accuracy: 0.8500 - val_loss: 0.3027 - val_accuracy: 0.8359

Epoch 00002: val_loss improved from 0.50788 to 0.30269, saving model to Nuclei_model.h5
Epoch 3/25
19/19 [==============================] - 88s 5s/step - loss: 0.2461 - accuracy: 0.8586 - val_loss: 0.2417 - val_accuracy: 0.8424

Epoch 00003: val_loss improved from 0.30269 to 0.24170, saving model to Nuclei_model.h5
Epoch 4/25
19/19 [==============================] - 91s 5s/step - loss: 0.1929 - accuracy: 0.8933 - val_loss: 0.1928 - val_accuracy: 0.9134

Epoch 00004: val_loss improved from 0.24170 to 0.19284, saving model to Nuclei_model.h5
Epoch 5/25
19/19 [==============================] - 92s 5s/step - loss: 0.1569 - accura

In [ ]:
# save the model
model.save("final_model/model.pb")

In [33]:
3600*12

43200

In [76]:
preds_train = model.predict(x_train[:int(x_train.shape[0] * 0.9)], verbose=1)
preds_val = model.predict(x_train[int(x_train.shape[0] * 0.9):], verbose=1)
preds_test = model.predict(x_test, verbose=1)

3/3 [==============================] - 2s 391ms/step


In [77]:
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

In [82]:
idx = randint(0, len(preds_val_t))
cv2.imshow('img', x_train[:int(x_train.shape[0] * 0.9)][idx])
cv2.waitKey(0)
cv2.destroyAllWindows()

In [187]:
# Perform a sanity check on some random validation samples
idx = random.randint(0, len(preds_val_t))
cv2.imshow('img', x_train[:int(x_train.shape[0] * 0.9)][idx])
cv2.waitKey(0)
y_train_ = y_train[int(y_train.shape[0]*0.9):][idx]
cv2.imshow('img0', np.uint8(np.where(y_train_ == True, 255, 0)))
cv2.waitKey(0)
pred_img = np.where(preds_val_t[idx] == True, 255, 0)
cv2.imshow('img1', np.uint8(pred_img))
cv2.waitKey(0)

cv2.destroyAllWindows()

In [179]:
cv2.destroyAllWindows()